In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import pickle
from collections import Counter
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from scipy.stats.mstats import gmean
import math

In [2]:
df_new = pd.read_stata("df_new.dta")

In [14]:
predictors = pickle.load(open("predictors.p", "rb")) + ['white', 'afam', 'hisp', 'asian', 'other']

In [31]:
len(predictors)

299

In [15]:
def calc_cw(y):
    # Calculate the weight of each letter grade to be used in the modeling fitting procedure: the weight is inversely proportional to the square root of the frequency of the letter grade in the training sample
    cw = Counter(y)
    class_weight = {k:np.sqrt(cw.most_common()[0][-1]/v, dtype=np.float32) for k,v in cw.items()}
    return class_weight # The output is a dictionary mapping letter grade to the corresponding weight

In [16]:
train_df = df_new[df_new.valid == 0]
test_df = df_new[df_new.valid == 1]

In [17]:
optimal_d = 16
optimal_n = 100
optimal_nf = 13
rf = RandomForestClassifier(n_estimators=optimal_n, criterion="entropy",
                            max_depth=optimal_d,
                            random_state=0, n_jobs=-1, max_features=optimal_nf,
                            class_weight = calc_cw(train_df.grad_6years))
rf.fit(train_df.loc[:,predictors], train_df.grad_6years)

RandomForestClassifier(bootstrap=True,
            class_weight={0.0: 1.0, 1.0: 1.3931639}, criterion='entropy',
            max_depth=16, max_features=13, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [21]:
xx = np.array(predictors)[np.argsort(rf.feature_importances_)[::-1]]
yy = rf.feature_importances_[np.argsort(rf.feature_importances_)[::-1]]
fi_table = pd.DataFrame({'predictor':xx, 'fi':yy})

In [30]:
fi_table.head()

,fi,predictor
0,0.079968,prop_comp
1,0.067962,cum_gpa
2,0.044348,prop_comp_sd
3,0.027934,term_gpa_sp1
4,0.026560,withdrawn_prop_comp


In [23]:
fi_table[fi_table.predictor == "afam"]

,fi,predictor
115,0.001396,afam


In [28]:
top10_predictors = pd.DataFrame({'predictor':xx, 'fi':yy}).iloc[:10,:]

In [29]:
top10_predictors.to_csv("top10_predictors_with_race.csv", index=False)

In [32]:
df2 = pd.read_stata("degree_completion_full_sample_orig_RF.dta")
df2 = df2.merge(test_df.loc[:,['vccsid','white','afam']])

In [33]:
top10_new = [p for p in top10 if p.endswith("1")]
top10_new

['prop_comp_sp1', 'term_gpa_fa1', 'term_gpa_sp1', 'prop_comp_fa1']

In [34]:
from statsmodels.stats.weightstats import CompareMeans,DescrStatsW

In [42]:
rows = []
for pp in top10_predictors:
    if pp in top10_new:
        p = pp + "_orig"
    else:
        p = pp
    x = np.array(df2.loc[:,p])
    white_indices = np.where(df2.white == 1)[0]
    nonwhite_indices = np.where(df2['afam'] == 1)[0]
    x_1 = x[white_indices]
    x_1 = x_1[~pd.isnull(x_1)]
    x_2 = x[nonwhite_indices]
    x_2 = x_2[~pd.isnull(x_2)]
    test_result = CompareMeans(DescrStatsW(x_1), DescrStatsW(x_2)).ztest_ind(alternative='two-sided', usevar='unequal')
    p_vals = str(test_result[1]).split("e")
    if len(p_vals) == 1:
        new_p_val = str(round(float(p_vals[0]),4))
    else:
        new_p_val = str(round(float(p_vals[0]),4)) + "e" + p_vals[1]
    white_mean = round(np.mean(x_1), 4)
    diff_in_mean = round(np.mean(x_2) - np.mean(x_1),4)
    if pp in top10_new:
        x_1 = x[white_indices]
        x_1 = pd.isnull(x_1).astype(int)
        x_2 = x[nonwhite_indices]
        x_2 = pd.isnull(x_2).astype(int)
        test_result = CompareMeans(DescrStatsW(x_1), DescrStatsW(x_2)).ztest_ind(alternative='two-sided', usevar='unequal')
        p_vals = str(test_result[1]).split("e")
        if len(p_vals) == 1:
            new_p_val_2 = str(round(float(p_vals[0]),4))
        else:
            new_p_val_2 = str(round(float(p_vals[0]),4)) + "e" + p_vals[1]
        white_mean_2 = round(np.mean(x_1), 4)
        diff_in_mean_2 = round(np.mean(x_2) - np.mean(x_1),4)
        rows.append((pp, white_mean, diff_in_mean, new_p_val, white_mean_2, diff_in_mean_2, new_p_val_2))
    else:
        rows.append((pp, white_mean, diff_in_mean, new_p_val))
table = pd.DataFrame(rows[:], columns=['predictor', "white_mean", 'diff_in_mean', 'p_value', 'white_mi_perc', 'diff_in_mi_perc', 'p_value_mi'])
table.loc[:,'p_value'] = table.p_value.astype(float).round(4)
table.loc[:,'p_value_mi'] = table.p_value_mi.astype(float).round(4)
table.round(4).to_csv("key_predictor_difference_{}.csv".format('afam'), index=False)